In [162]:
import copy      

print('------------------------------------------')
print('2024, Day 6 Part 1')
print('------------------------------------------')
UP = 'UP'
RIGHT = 'RIGHT'
LEFT = 'LEFT'
DOWN = 'DONW'
OFF_GRID = 'OFF_GRID'
OCCUPIED = 'OCCUPIED'
LOOP_DETECTED = 'LOOP DETECTED'
MARK = 'X'
OBSTACLE = '#'
START_MARKER = '^'
PLACED_OBSTACLE = 'O'
UNOCCUPIED = '.'
FREE = 'FREE'
curr_path = []
start_pos = None
obstacle_pos = { 'x': 0, 'y': -1 }
orig_path = []

def read_map():
    fpath = 'C:/Users/akamath/Documents/AOC2024/input6.txt'
    file = open(fpath)
    path = []
    while True:
        line = file.readline().strip()
        if not line:
            break;
        path.append(list(line))
    file.close()
    return path
    
# convert back to string and print on separate line for readability    
def print_it():
    l = []
    cnt = 0
    for items in path:
        s = ''.join(items)
        l.append(s)
    for items in l:
        cnt += items.count(MARK)
        print(items)
    print('Count:', cnt)
    print('Loop count:', loop_cnt)
    
def reset_path():
    path = []
    path = copy.deepcopy(orig_path)
    
def find_start(path):
    for i in range(len(path)):
        ind = None
        try:
            ind = path[i].index('^')
            dir = UP
        except ValueError:
            pass
        if ind:
            break
    pos = {
        'x': i,
        'y': ind
    }
    start_pos = pos
    return pos

def off_grid(pos, d):
    if d == UP:
        if pos['x'] == 0:
            return True
    elif d == DOWN:
        if pos['x'] == len(path)-1:
            return True
    elif d == RIGHT:
        if pos['y'] == len(path[0])-1:
            return True
    else:
        if pos['y'] == 0:
            return True
    return False
        
def get_value(pos):
    return path[pos['x']][pos['y']]
    
def get_up_value(pos):
    return path[pos['x']-1][pos['y']]
                     
def get_left_value(pos):
    return path[pos['x']][pos['y']-1]
                     
def get_down_value(pos):
    return path[pos['x']+1][pos['y']]

def get_right_value(pos):
    return path[pos['x']][pos['y']+1]

def up_occupied(pos):
    if get_up_value(pos) == OBSTACLE or get_up_value(pos) == PLACED_OBSTACLE:
        return OCCUPIED
    return FREE

def is_occupied(pos):
    if get_value(pos) == OBSTACLE or get_value(pos) == PLACED_OBSTACLE or get_value(pos) == START_MARKER:
        return OCCUPIED
    return FREE
    
def right_occupied(pos):
    if get_right_value(pos) == OBSTACLE or get_right_value(pos) == PLACED_OBSTACLE:
        return OCCUPIED
    return FREE

def down_occupied(pos):
    if get_down_value(pos) == OBSTACLE or get_down_value(pos) == PLACED_OBSTACLE:
        return OCCUPIED
    return FREE

def left_occupied(pos):
    if get_left_value(pos) == OBSTACLE or get_left_value(pos) == PLACED_OBSTACLE:
        return OCCUPIED
    return FREE
                     
def occupied(pos, d):
    if off_grid(pos,d):
        return OFF_GRID
    if d == UP:
        return up_occupied(pos)
    if d == LEFT:
        return left_occupied(pos)
    if d == RIGHT:
        return right_occupied(pos)
    if d == DOWN:
        return down_occupied(pos) 
    return FREE

def check_dead_end(pos):
    if left_occupied(pos) == OCCUPIED and up_occupied(pos) == OCCUPIED and right_occupied(pos) == OCCUPIED and down_occupied(pos) == OCCUPIED:
        return True
    return False

def check_loop(start, pos, d):
    if pos == start and d == UP:
        return True
    return False

def get_next_pos(pos):
    starti = pos['y']
    startj = pos['x']
    new_pos = None
    if starti+1 < len(path[0]):
        starti += 1
        new_pos = { 'x': startj, 'y': starti }
    elif startj+1 < len(path):
        starti = 0
        startj += 1
        new_pos = { 'x': startj, 'y': starti }
    return new_pos
        
def place_obstacle(pos=None):
    if pos:
        #Clear this position
        set_value(pos, UNOCCUPIED)   
    else:
        pos = {'x': 0, 'y': -1 }
    curr_pos = { 'x': pos.get('x'), 'y': pos.get('y') }

    pos = get_next_pos(curr_pos)
    if pos:
        while is_occupied(pos) == OCCUPIED:
            pos = get_next_pos(pos)
            if not pos:
                break
    if pos:
        set_value(pos, 'O')
    return pos

def set_value(pos, val):
    path[pos['x']][pos['y']] = val
    
def move_up(pos):
    status = occupied(pos, UP)
    while status == FREE:
        pos['x'] = pos['x']-1
        if (pos, UP) in path_taken:
            return LOOP_DETECTED, pos
        p = copy.deepcopy(pos)
        path_taken.append((p, UP))
        set_value(pos, MARK)
        status = occupied(pos, UP)
    return status, pos

def move_right(pos):
    status = occupied(pos, RIGHT)
    while status == FREE:
        pos['y'] = pos['y']+1
        if (pos, RIGHT) in path_taken:
            return LOOP_DETECTED, pos
        p = copy.deepcopy(pos)
        path_taken.append((p, RIGHT))
        set_value(pos, MARK)
        status = occupied(pos, RIGHT)
    return status, pos

def move_down(pos):
    status = occupied(pos, DOWN)
    while status == FREE:
        pos['x'] = pos['x']+1
        if (pos, DOWN) in path_taken:
            return LOOP_DETECTED, pos
        p = copy.deepcopy(pos)
        path_taken.append((p, DOWN))
        set_value(pos, MARK)
        status = occupied(pos, DOWN) 
    return status, pos

def move_left(pos):
    status = occupied(pos, LEFT)
    while status == FREE:
        pos['y'] = pos['y']-1
        if (pos, LEFT) in path_taken:
            return LOOP_DETECTED, pos
        p = copy.deepcopy(pos)
        path_taken.append((p, LEFT))
        set_value(pos, MARK)
        status = occupied(pos, LEFT)   
    return status, pos

def start_patrol(pos, patrol):
    #Mark the starting position
    start_pos = { 'x': pos.get('x'), 'y': pos.get('y') }
    set_value(pos, MARK);
    curr_dir = UP
    pos = copy.deepcopy(start_pos)
    iter = 0
    while True:
        status, pos = move_up(pos)
        if status == OFF_GRID or status == LOOP_DETECTED:
            break
        status, pos = move_right(pos)
        if status == OFF_GRID or status == LOOP_DETECTED:
            break
        status, pos = move_down(pos)
        if status == OFF_GRID or status == LOOP_DETECTED:
            break
        status, pos = move_left(pos)
        if status == OFF_GRID or status == LOOP_DETECTED:
            break
    if status == LOOP_DETECTED:
        print('LOOP DETECTED')
        return 1
    return 0
         
path = read_map()
orig_path = copy.deepcopy(path)
start_pos = find_start(path)
obstacle_pos = place_obstacle()
loop_cnt = 0
cnt = 0
while(True):
    path_taken = []
    loop_cnt += start_patrol(start_pos, cnt)
    path = copy.deepcopy(orig_path)
    obstacle_pos = place_obstacle(obstacle_pos)
    if not obstacle_pos:
        break
    cnt += 1
print_it()


------------------------------------------
2024, Day 6 Part 1
------------------------------------------
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED
LOOP DETECTED